<h1>Содержание<span class="tocSkip"></span></h1>
<li>Подготовка данных</li>
<li>Обучение и проверка модели</li>
<li>Подготовка к расчёту прибыли</li>
<li>Расчёт прибыли и рисков</li>
<li>Вывод</li>


# Выбор локации для скважины

В рамках данного проекта нужно решить, где бурить новую скважину.

Для анализа предоставлены объемы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль и проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Каждая таблица - данные геологоразведки по одному региону**

**Таблицы содержат следующие поля:**

* `id` - уникальный идентификатор скважины

* `f0`,`f1`,`f2` - признаки точек

* `product` - объём запасов в скважине (тыс. баррелей)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import random
import numpy as np

In [2]:
geo_data_1 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
geo_data_2 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
geo_data_3 = pd.read_csv('/datasets/geo_data_2.csv')
geo_data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [7]:
geo_data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Каждая таблицы содержит 100к записей, пропусков нет. Для дальнейшей работы столбец с айдишниками нам не понадобится, дропнем.**

In [8]:
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)
geo_data_3 = geo_data_3.drop(['id'], axis=1)

## Обучение и проверка модели

**Разобьем данные на валидационную и обучающую выборки:**

In [9]:
features_1 = geo_data_1.drop(['product'], axis=1)
features_2 = geo_data_2.drop(['product'], axis=1)
features_3 = geo_data_3.drop(['product'], axis=1)

target_1 = geo_data_1['product']
target_2 = geo_data_2['product']
target_3 = geo_data_3['product']

In [10]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=0)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=0)

features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=0)

In [11]:
print(features_train_1.shape)
print(features_valid_2.shape)
print(target_train_3.shape)

(75000, 3)
(25000, 3)
(75000,)


**Обучим модели:**

**Первый регион**

In [12]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predictions_valid_1 = model.predict(features_valid_1)
mse = mean_squared_error(target_valid_1, predictions_valid_1)
rmse_1 = mse**0.5


print('Предсказанный средний запас:', predictions_valid_1.mean())
print('RMSE:', rmse_1)

Предсказанный средний запас: 92.27144852242301
RMSE: 37.48100896950594


**Второй регион**

In [13]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predictions_valid_2 = model.predict(features_valid_2)
mse = mean_squared_error(target_valid_2, predictions_valid_2)
rmse_2 = mse**0.5


print('Предсказанный средний запас:', predictions_valid_2.mean())
print('RMSE:', rmse_2)

Предсказанный средний запас: 69.15162398290752
RMSE: 0.8872573052219331


**Третий регион**

In [14]:
model = LinearRegression()
model.fit(features_train_3, target_train_3)
predictions_valid_3 = model.predict(features_valid_3)

mse = mean_squared_error(target_valid_3, predictions_valid_3)
rmse_3 = mse**0.5


print('Предсказанный средний запас:', predictions_valid_3.mean())
print('RMSE:', rmse_3)

Предсказанный средний запас: 94.70753129105672
RMSE: 40.31290686044374


**По итогу можно отметить, что средний запас сырья в первом и третьем регоинах значительно выше, чем во втором. При этом, ошибка предсказания в первом и третьем регионах крайне высокая, у второго же региона она практически отсутствует.** 

**Вероятно, в данных второго региона модель нашла довольно сильную корреляцию между признаками (или одним из признаков) и итоговым количеством сырья.**

## Подготовка к расчёту прибыли

In [15]:
wells_to_explore = 500
wells_to_develop = 200
budget = 10**10 #10 млрд
income_per_unit = 450 * 1000 
break_even = budget / income_per_unit 
break_even_per_well = break_even / wells_to_develop

print('Необходимый объём для безубыточной разработки по всему региону:', math.ceil(break_even))
print()
print('Необходимый объём для безубыточной разработки по одной скважине:', math.ceil(break_even_per_well))

Необходимый объём для безубыточной разработки по всему региону: 22223

Необходимый объём для безубыточной разработки по одной скважине: 112


**Необходимый объём для безубыточности по одной скважине превышает предсказанное среднее значение по каждому из регионов, но стоит учитывать, что мы будем выбирать 200 лучших скважин, а среднее считалось по всем имеющимся скважинам.**

## Расчёт прибыли и рисков 

**Далее отберём лучшие 200 скважин, просуммируем объём сырья и посчитаем прибыль.**

In [16]:
predictions_valid_1 = pd.Series(predictions_valid_1).reset_index(drop=True)
predictions_valid_2 = pd.Series(predictions_valid_2).reset_index(drop=True)
predictions_valid_3 = pd.Series(predictions_valid_3).reset_index(drop=True)

target_valid_1 = pd.Series(target_valid_1).reset_index(drop=True)
target_valid_2 = pd.Series(target_valid_2).reset_index(drop=True)
target_valid_3 = pd.Series(target_valid_3).reset_index(drop=True)

def profit_counter(predicted_product, real_product):
    
    predicted_best_wells = predicted_product.sample(
        wells_to_explore, replace=True, random_state=0).sort_values(
        ascending=False).head(wells_to_develop)
    
    real_best_wells = real_product.loc[predicted_best_wells.index]
    real_best_wells_sum = math.floor(real_best_wells.sum())
    real_total_profit = math.floor(real_best_wells_sum * income_per_unit) - budget
    
    print('Предсказанный объём 200 лучших скважин:', real_best_wells_sum) 
    print('Потенциальная прибыль:', real_total_profit)
   

In [17]:
print('Первый регион:')
profit_counter(predictions_valid_1, target_valid_1)
print()
print('Второй регион:')
profit_counter(predictions_valid_2, target_valid_2)
print()
print('Третий регион:')
profit_counter(predictions_valid_3, target_valid_3)

Первый регион:
Предсказанный объём 200 лучших скважин: 23134
Потенциальная прибыль: 410300000

Второй регион:
Предсказанный объём 200 лучших скважин: 23103
Потенциальная прибыль: 396350000

Третий регион:
Предсказанный объём 200 лучших скважин: 22556
Потенциальная прибыль: 150200000


**По итогу видим, что все регионы прошли точку безубыточности в 22 223 тыс. баррелей.**

**Далее (чтобы исключить случайность при выборе скважин) посчитаем прибыль и риски техникой Bootstrap, для этого немного поправим возврат функции profit_counter**

In [18]:
state = np.random.RandomState(0)

def profit_counter(predicted_product, real_product):
    
    predicted_best_wells = predicted_product.sample(
        wells_to_explore, replace=True, random_state=state).sort_values(
        ascending=False).head(wells_to_develop)
    
    real_best_wells = real_product.loc[predicted_best_wells.index]
    real_best_wells_sum = math.floor(real_best_wells.sum())
    real_total_profit = math.floor(real_best_wells_sum * income_per_unit) - budget
    
    return real_total_profit

**Первый регион**

In [19]:
values_1 = []

for i in range(1000):
    values_1.append(profit_counter(predictions_valid_1, target_valid_1))
    
values_1 = pd.Series(values_1)
lower_1 = values_1.quantile(0.025)
higher_1 = values_1.quantile(0.975)
mean_1 = values_1.mean()
loss_probability_1 = values_1[values_1 < 0].count() / len(values_1) * 100

print('Вероятность убытка:', loss_probability_1, '%')
print('Средняя прибыль:', math.floor(mean_1))
print('2,5%-квантиль:', math.floor(lower_1))
print('97,5%-квантиль:', math.floor(higher_1))

Вероятность убытка: 3.9 %
Средняя прибыль: 442646900
2,5%-квантиль: -49262500
97,5%-квантиль: 921061250


**Второй регион**

In [20]:
values_2 = []

for i in range(1000):
    values_2.append(profit_counter(predictions_valid_2, target_valid_2))
    
values_2 = pd.Series(values_2)
lower_2 = values_2.quantile(0.025)
higher_2 = values_2.quantile(0.975)
mean_2 = values_2.mean()
loss_probability_2 = values_2[values_2 < 0].count() / len(values_2) * 100

print('Вероятность убытка:', loss_probability_2, '%')
print('Средняя прибыль:', math.floor(mean_2))
print('2,5%-квантиль:', math.floor(lower_2))
print('97,5%-квантиль:', math.floor(higher_2))

Вероятность убытка: 0.8 %
Средняя прибыль: 469305800
2,5%-квантиль: 84860000
97,5%-квантиль: 850478749


**Третий регион**

In [21]:
values_3 = []

for i in range(1000):
    values_3.append(profit_counter(predictions_valid_3, target_valid_3))
    
values_3 = pd.Series(values_3)
lower_3 = values_3.quantile(0.025)
higher_3 = values_3.quantile(0.975)
mean_3 = values_3.mean()
loss_probability_3 = values_3[values_3 < 0].count() / len(values_3) * 100

print('Вероятность убытка:', loss_probability_3, '%')
print('Средняя прибыль:', math.floor(mean_3))
print('2,5%-квантиль:', math.floor(lower_3))
print('97,5%-квантиль:', math.floor(higher_3))

Вероятность убытка: 6.9 %
Средняя прибыль: 384194600
2,5%-квантиль: -151243750
97,5%-квантиль: 916606250


**По итогу проверку на риск прошел только второй регион, только там вероятность уйти в минус меньше 2.5%. Более того, средняя прибыль в данном регионе выше, чем в двух других. Также стоит заметить, что предел доверительного интервала у второго региона самый низкий, а у третьего региона (с недопустимым уровнем риска) самый высокий**

## Вывод

**В рамках данного проекта я:**
* Обучил модель линейной регрессии и спрогнозировал количество баррелей в скважинах трёх регионов. Первый и третий регион выглядели перспективнее второго
* Выбрал случайные 500 скважин для каждого региона и рассчитал по ним прибыль
* Провел тысячу экспериментов с разными выборками из 500 скважин для снижения влияния случайности
* Выяснил, что под заданные бизнесом параметры риска подходит только второй регион